In [25]:
from IPython.core.display import display, HTML #für eine Breitansicht des Jupyter Notebooks
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

train = pd.read_csv('/Users/patrickbaongu/Desktop/Titanic/train.csv')
test = pd.read_csv('/Users/patrickbaongu/Desktop/Titanic/test.csv')

train_test_data=[train, test]

In [26]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [27]:
drop_features = ['Name','SibSp', 'Parch', 'Ticket', 'Cabin','Embarked', 'Pclass']

for data in train_test_data:

    
    data['Sex'] = data['Sex'].map({'male':0, 'female':1})
    
    data['Age'].fillna(data['Age'].mean(), inplace = True)
    data['Age'] = pd.cut(data['Age'], bins = np.linspace(0,100,5) ,labels = np.linspace(0,1,4), include_lowest = True) 
    
    data['Fare'].replace(0,np.nan)
    data['Fare'].fillna(data['Fare'].mean(), inplace = True)
    
    data['Fare'] = pd.cut(data['Fare'], bins = [0,15,30,45,60,data['Fare'].max()], labels = np.linspace(0,1,5), include_lowest = True)
    
    data['Family Size'] = 1 + data['SibSp'] + data['Parch']
    data['Family Size'] = pd.cut(data['Family Size'], bins = [0,1,4, data['Family Size'].max()], labels = np.linspace(0,1,3), include_lowest = True)

    
    data.drop(drop_features, axis = 1 ,inplace = True)
    
    

train.drop('PassengerId', axis = 1, inplace = True)
train_data = train.drop('Survived', axis = 1)
labels = train['Survived']

In [28]:
kfold = KFold(n_splits = 8, shuffle = True, random_state = 42)
models = [KNeighborsClassifier(n_neighbors = 12), DecisionTreeClassifier(), RandomForestClassifier(), GaussianNB(), SVC(gamma = auto)]
scores = []
for model in models:

    cv_score = cross_val_score(estimator = model, X = train_data, y = labels, cv = kfold, n_jobs =1, scoring = 'accuracy')
    rounded = round(np.mean(cv_score),4)
    scores.append(rounded)
max_idx = np.argmax(scores)
scores

NameError: name 'auto' is not defined

In [17]:
test_data = test.drop('PassengerId', axis =1).copy()
clf = models[max_idx].fit(train_data, labels)
y_pred = clf.predict(test_data)

In [18]:
submission = pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':y_pred})
submission.to_csv('submission.csv', index = False)
submission = pd.read_csv('submission.csv')